In [12]:
from qwikidata.sparql import return_sparql_query_results

query = """
SELECT ?item WHERE {
  SERVICE wikibase:mwapi {
      bd:serviceParam wikibase:endpoint "www.wikidata.org";
                      wikibase:api "EntitySearch";
                      mwapi:search "Kıbrıs Fatihi";
                      mwapi:language "en".
      ?item wikibase:apiOutputItem mwapi:item.
      ?num wikibase:apiOrdinal true.
  }
  ?item (wdt:P279|wdt:P31) ?type
} ORDER BY ASC(?num) LIMIT 20
"""
res = return_sparql_query_results(query)



In [13]:
res

{'head': {'vars': ['item']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q192634'}}]}}